# TaskTree Tutorial

In this tutorial we will walk through the capabilities of task trees in pycram.

First we have to import the necessary functionality from pycram.

In [1]:
from pycram.worlds.bullet_world import BulletWorld, Object
from pycram.robot_descriptions import robot_description
import pycram.tasktree
from pycram.datastructures.enums import Arms, ObjectType
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.process_module import simulated_robot
from pycram.designators.object_designator import *
from pycram.datastructures.pose import Pose
from pycram.datastructures.enums import ObjectType, WorldMode
import anytree
import pycram.plan_failures

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Failed to import Giskard messages, the real robot will not be available
Could not import RoboKudo messages, RoboKudo interface could not be initialized
pybullet build time: Nov 28 2023 23:51:11


Next we will create a bullet world with a PR2 in a kitchen containing milk and cereal.

In [2]:
world = BulletWorld(mode=WorldMode.GUI)
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")
milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([1.3, 1, 0.9]))
cereal = Object("cereal", ObjectType.BREAKFAST_CEREAL, "breakfast_cereal.stl", pose=Pose([1.3, 0.7, 0.95]))
milk_desig = ObjectDesignatorDescription(names=["milk"])
cereal_desig = ObjectDesignatorDescription(names=["cereal"])
robot_desig = ObjectDesignatorDescription(names=["pr2"]).resolve()
kitchen_desig = ObjectDesignatorDescription(names=["kitchen"])

Scalar element defined multiple times: limit
Unknown tag "rgba_color" in /robot[@name='milk_object']/link[@name='milk_main']/visual[1]/material[@name='white']
Scalar element defined multiple times: limit
Unknown tag "rgba_color" in /robot[@name='cereal_object']/link[@name='cereal_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='milk_object']/link[@name='milk_main']/visual[1]/material[@name='white']
Unknown tag "rgba_color" in /robot[@name='cereal_object']/link[@name='cereal_main']/visual[1]/material[@name='white']


Finally, we create a plan where the robot parks his arms, walks to the kitchen counter and picks the cereal and places it on the table. Then we execute the plan.

In [3]:
@pycram.tasktree.with_tree
def plan():
    with simulated_robot:
        ParkArmsActionPerformable(Arms.BOTH).perform()
        MoveTorsoAction([0.3]).resolve().perform()
        pickup_pose = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot_desig).resolve()
        pickup_arm = pickup_pose.reachable_arms[0]
        NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()
        PickUpAction(object_designator_description=cereal_desig, arms=[pickup_arm], grasps=["front"]).resolve().perform()
        ParkArmsAction([Arms.BOTH]).resolve().perform()

        place_island = SemanticCostmapLocation("kitchen_island_surface", kitchen_desig.resolve(),
                                           cereal_desig.resolve()).resolve()

        place_stand = CostmapLocation(place_island.pose, reachable_for=robot_desig, reachable_arm=pickup_arm).resolve()

        NavigateAction(target_locations=[place_stand.pose]).resolve().perform()

        PlaceAction(cereal_desig, target_locations=[place_island.pose], arms=[pickup_arm]).resolve().perform()

        ParkArmsAction([Arms.BOTH]).resolve().perform()

        ParkArmsActionPerformable(Arms.BOTH).perform()

plan()


[INFO] [1720609552.729942]: Ontology [http://www.ease-crc.org/ont/SOMA-HOME.owl#]'s name: SOMA-HOME has been loaded
[INFO] [1720609552.730620]: - main namespace: SOMA-HOME
[INFO] [1720609552.731192]: - loaded ontologies:
[INFO] [1720609552.731771]: http://www.ease-crc.org/ont/SOMA-HOME.owl#
[INFO] [1720609552.732239]: http://www.ease-crc.org/ont/DUL.owl#
[INFO] [1720609552.732606]: http://www.ease-crc.org/ont/SOMA.owl#
[INFO] [1720609553.680282]: Waiting for IK service: /pr2_left_arm_kinematics/get_ik


Now we get the task tree from its module and render it. Rendering can be done with any render method described in the anytree package. We will use ascii rendering here for ease of displaying.

In [4]:
tt = pycram.tasktree.task_tree
print(anytree.RenderTree(tt))

NoOperation
└── NoOperation
    ├── ParkArmsActionPerformable
    ├── MoveTorsoActionPerformable
    ├── NavigateActionPerformable
    │   └── MoveMotion
    ├── PickUpActionPerformable
    │   ├── MoveTCPMotion
    │   ├── MoveGripperMotion
    │   ├── MoveTCPMotion
    │   ├── MoveGripperMotion
    │   └── MoveTCPMotion
    ├── ParkArmsActionPerformable
    ├── NavigateActionPerformable
    │   └── MoveMotion
    ├── PlaceActionPerformable
    │   ├── MoveTCPMotion
    │   ├── MoveGripperMotion
    │   └── MoveTCPMotion
    ├── ParkArmsActionPerformable
    └── ParkArmsActionPerformable


As we see every task in the plan got recorded correctly. It is noticeable that the tree begins with a NoOperation node. This is done because several, not connected, plans that get executed after each other should still appear in the task tree. Hence, a NoOperation node is the root of any tree. If we re-execute the plan we would see them appear in the same tree even though they are not connected.

In [5]:
world.reset_world()
plan()
print(anytree.RenderTree(tt))

NoOperation
├── NoOperation
│   ├── ParkArmsActionPerformable
│   ├── MoveTorsoActionPerformable
│   ├── NavigateActionPerformable
│   │   └── MoveMotion
│   ├── PickUpActionPerformable
│   │   ├── MoveTCPMotion
│   │   ├── MoveGripperMotion
│   │   ├── MoveTCPMotion
│   │   ├── MoveGripperMotion
│   │   └── MoveTCPMotion
│   ├── ParkArmsActionPerformable
│   ├── NavigateActionPerformable
│   │   └── MoveMotion
│   ├── PlaceActionPerformable
│   │   ├── MoveTCPMotion
│   │   ├── MoveGripperMotion
│   │   └── MoveTCPMotion
│   ├── ParkArmsActionPerformable
│   └── ParkArmsActionPerformable
└── NoOperation
    ├── ParkArmsActionPerformable
    ├── MoveTorsoActionPerformable
    ├── NavigateActionPerformable
    │   └── MoveMotion
    ├── PickUpActionPerformable
    │   ├── MoveTCPMotion
    │   ├── MoveGripperMotion
    │   ├── MoveTCPMotion
    │   ├── MoveGripperMotion
    │   └── MoveTCPMotion
    ├── ParkArmsActionPerformable
    ├── NavigateActionPerformable
    │   └── MoveMotion
 

Projecting a plan in a new environment with its own task tree that only exists while the projected plan is running can be done with the ``with`` keyword. When this is done, both the bullet world and task tree are saved and new, freshly reset objects are available. At the end of a with block the old state is restored. The root for such things is then called ``simulation()``.

In [6]:
with pycram.tasktree.SimulatedTaskTree() as stt:
    print(anytree.RenderTree(pycram.tasktree.task_tree))
print(anytree.RenderTree(pycram.tasktree.task_tree))

NoOperation
NoOperation
├── NoOperation
│   ├── ParkArmsActionPerformable
│   ├── MoveTorsoActionPerformable
│   ├── NavigateActionPerformable
│   │   └── MoveMotion
│   ├── PickUpActionPerformable
│   │   ├── MoveTCPMotion
│   │   ├── MoveGripperMotion
│   │   ├── MoveTCPMotion
│   │   ├── MoveGripperMotion
│   │   └── MoveTCPMotion
│   ├── ParkArmsActionPerformable
│   ├── NavigateActionPerformable
│   │   └── MoveMotion
│   ├── PlaceActionPerformable
│   │   ├── MoveTCPMotion
│   │   ├── MoveGripperMotion
│   │   └── MoveTCPMotion
│   ├── ParkArmsActionPerformable
│   └── ParkArmsActionPerformable
└── NoOperation
    ├── ParkArmsActionPerformable
    ├── MoveTorsoActionPerformable
    ├── NavigateActionPerformable
    │   └── MoveMotion
    ├── PickUpActionPerformable
    │   ├── MoveTCPMotion
    │   ├── MoveGripperMotion
    │   ├── MoveTCPMotion
    │   ├── MoveGripperMotion
    │   └── MoveTCPMotion
    ├── ParkArmsActionPerformable
    ├── NavigateActionPerformable
    │   └── 

Task tree can be manipulated with ordinary anytree manipulation. If we for example want to discard the second plan, we would write

In [7]:
tt.root.children = (tt.root.children[0],)
print(anytree.RenderTree(tt, style=anytree.render.AsciiStyle()))

NoOperation
+-- NoOperation
    |-- ParkArmsActionPerformable
    |-- MoveTorsoActionPerformable
    |-- NavigateActionPerformable
    |   +-- MoveMotion
    |-- PickUpActionPerformable
    |   |-- MoveTCPMotion
    |   |-- MoveGripperMotion
    |   |-- MoveTCPMotion
    |   |-- MoveGripperMotion
    |   +-- MoveTCPMotion
    |-- ParkArmsActionPerformable
    |-- NavigateActionPerformable
    |   +-- MoveMotion
    |-- PlaceActionPerformable
    |   |-- MoveTCPMotion
    |   |-- MoveGripperMotion
    |   +-- MoveTCPMotion
    |-- ParkArmsActionPerformable
    +-- ParkArmsActionPerformable


We can now re-execute this (modified) plan by executing the leaf in pre-ordering iteration using the anytree functionality. This will not append the re-execution to the task tree.

In [10]:
world.reset_world()
with simulated_robot:
    [node.code.execute() for node in tt.root.leaves]
print(anytree.RenderTree(pycram.tasktree.task_tree, style=anytree.render.AsciiStyle()))

AttributeError: 'TaskTreeNode' object has no attribute 'execute'

Nodes in the task tree contain additional information about the status and time of a task.

In [11]:
print(pycram.tasktree.task_tree.children[0])

Code: NoOperation 
 Status: 2024-07-10 13:05:50.604755 
 start_time: TaskStatus.SUCCEEDED 
 end_time: 2024-07-10 13:06:01.899865 
 


The task tree can also be reset to an empty one by invoking

In [12]:
pycram.tasktree.reset_tree()
print(anytree.RenderTree(pycram.tasktree.task_tree, style=anytree.render.AsciiStyle()))

NoOperation


If a plan fails using the PlanFailure exception, the plan will not stop. Instead, the error will be logged and saved in the task tree as a failed subtask. First let's create a simple failing plan and execute it.

In [13]:
@pycram.tasktree.with_tree
def failing_plan():
    raise pycram.plan_failures.PlanFailure("Oopsie!")

try:
    failing_plan()
except pycram.plan_failures.PlanFailure as e:
    print(e)

Oopsie!


We can now investigate the nodes of the tree, and we will see that the tree indeed contains a failed task.

In [14]:
print(anytree.RenderTree(pycram.tasktree.task_tree, style=anytree.render.AsciiStyle()))
print(pycram.tasktree.task_tree.children[0])

NoOperation
+-- NoOperation
Code: NoOperation 
 Status: 2024-07-10 13:09:51.978222 
 start_time: TaskStatus.FAILED 
 end_time: 2024-07-10 13:09:51.978497 
 


In [13]:
world.exit()

In [33]:
with simulated_robot:
    world.reset_world()
    MoveTorsoAction([0.2]).resolve().perform()
    pickup_pose = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot_desig).resolve()
    print(pickup_pose.pose)
    NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()
    pickup_pose = CostmapLocation(target=milk_desig.resolve(), reachable_for=robot_desig).resolve()
    print(pickup_pose.pose)
    NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()
    arm = pickup_pose.reachable_arms[1]
    PickUpAction(object_designator_description=milk_desig,arms= [arm],grasps= ["front"]).resolve().perform()

header: 
  seq: 0
  stamp: 
    secs: 1720610659
    nsecs: 736824750
  frame_id: "map"
pose: 
  position: 
    x: 0.48
    y: 0.6599999999999999
    z: 0.0
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.024368513240484207
    w: 0.9997030436896991
header: 
  seq: 0
  stamp: 
    secs: 1720610665
    nsecs: 383566141
  frame_id: "map"
pose: 
  position: 
    x: 0.8600000000000001
    y: 1.54
    z: 0.0
  orientation: 
    x: -0.0
    y: 0.0
    z: 0.4291427785934214
    w: -0.9032366664286375
